<a href="https://colab.research.google.com/github/ju-vargas/Classificacao-e-Pesquisa-de-Dados/blob/main/labCPD_03_CPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disciplina de Classificação e Pesquisa de Dados
 
# Laboratório #3 - Final
 
### Implementação e comparação (em C++) dos principais algoritmos de classificação por seleção, intercalação e lineares.

## Exercício I: algoritmos de seleção (Heapsort e relacionados)

Utilizando a versão padrão do `heapsort` que você implementou na aula sobre "**Classificação por seleção: parte 2**", faça uma análise de desempenho dele para arrays de tamanho 100, 1000 e 10000, tanto crescentes quanto decrescentes. Registre o tempo de processamento, a quantidade de trocas e a quantidade de comparaçoes.

Se você não elaborou o algoritmo naquela ocasião, implemente-o agora. Não se esqueça de que ele é um algoritmo local e que usa funções auxiliares (as quais estão disponíveis no material daquela aula). Você pode usar suas próprias funções ou estruturá-lo de outra forma, se desejar, mas ele deve seguir as mesmas regras e preceitos.

Observação: naquela aula não foi solicitado o encaminhamento dos exercícios. Haviam outras tarefas e comparações. Se você desejar um feedback sobre elas, encaminhe suas soluções ao final deste arquivo.

In [ ]:
%%writefile hs_test.cpp
// Escreva aqui seu algoritmo

Writing hs_test.cpp


In [ ]:
%%script bash
g++ hs_test.cpp -o hs_test

In [ ]:
%%script bash
./hs_test

## Exercício II: algoritmos de intercalação

A seguir você encontra uma **versão do algoritmo de merge** e um exemplo de sua aplicação.

Depois, há vários **algoritmos a implementar**, a saber:
1. `two_way_merge`. 
2. `multi_way_merge`.
2. `merge_sort`.

Implemente-os conforme o comportamento esperado deles. Depois, teste-os com arrays criados por você. Não se esqueça que `two_way_merge` e `multi_way_merge` trabalham em cima de arrays previamente ordenados. Já o `merge_sort` aceita qualquer tipo de array. 

In [ ]:
%%writefile sorting4.cpp
#include<iostream>
#include<tuple>
#include<string>
#include<vector>
#include<unordered_map>
#include<bits/stdc++.h>
#include<map>

using namespace std;

// Definições e variáveis globais necessárias
typedef int element_t;                                                          // tipo do elemento
typedef vector<element_t> array_t;                                              // tipo do array
typedef tuple<int, int, double> loginfo_t;                                      // armazena informações de desempenho <trocas, comparações, tempo em ms>

void merge(const array_t&, const array_t&, array_t&, loginfo_t&);
void two_way_merge(const vector<array_t>, array_t&, loginfo_t&);
void multi_way_merge(const vector<array_t>, array_t&, loginfo_t&);
void mergesort(array_t&, loginfo_t&);
void swap(element_t* n1, element_t* n2);
void merge_aux(array_t&, int, int, int, loginfo_t&);
void mergesort_aux(array_t&, int, int, loginfo_t&);

int main(void){
  loginfo_t loginfo;
  int qtd;
  array_t array1 = {1,2,4,6,7,8,15,22};
  array_t array2 = {1,2,3,4,5,6,7,8,25,27,35,44};
  array_t array3, array4;
  array_t array5 = {10,9,8,7,6,5,4,3,2,1};

  for(auto e:array1) cout << e << " ";  
  cout << endl;

  for(auto e:array2) cout << e << " ";  
  cout << endl;

  merge(array1, array2, array3, loginfo);
  
  for(auto e:array3) cout << e << " ";  
  cout << endl;

  vector<array_t> arrays = { array1, array2, array3};
  two_way_merge(arrays, array4, loginfo);
  for(auto e:array4) cout << e << " ";  
  cout << endl;

  multi_way_merge(arrays, array4, loginfo);
  for(auto e:array4) cout << e << " ";  
  cout << endl;

  mergesort(array5, loginfo);
  for(auto e:array5) cout << e << " ";  
  cout << endl;

  // TODO: mostrar log de operações

  return 0;
}

// ################################################
// Algoritmos de intercalação
// ################################################
void merge(const array_t& array1, const array_t& array2, array_t& array_final, loginfo_t& loginfo){
    int i = 0, j = 0, trocas = 0, comparacoes = 0;
    int qtd_a1 = array1.size();
    int qtd_a2 = array2.size();
    
    bool elementos = true;
    array_final.clear();    
    while(i<qtd_a1 && j<qtd_a2){
       if(array1[i] <= array2[j])
          array_final.push_back(array1[i++]);          
       else
          array_final.push_back(array2[j++]);       
    }
    
    if(j<qtd_a2 && i>=qtd_a1)                                                   // array 1 terminou
       for(auto e=j;e<qtd_a2;e++)
          array_final.push_back(array2[e]);
    
    if(i<qtd_a1 && j>=qtd_a2)                                                   // array 2 terminou
       for(auto e=i;e<qtd_a1;e++)
          array_final.push_back(array1[e]);

    // TODO: atualizar loginfo
}

// Recebe uma lista de arrays e intercala-os 2 a 2
// retorna um array com o resultado da intercalação
void two_way_merge(const vector<array_t> arrays, array_t& array_final, loginfo_t& loginfo){   
  // TODO: implementar
}

void mergesort(array_t& array, loginfo_t& loginfo){
  // TODO: implementar
}

// Recebe uma lista de arrays e intercala-os usando estrutura similar a heap-min
void multi_way_merge(const vector<array_t> arrays, array_t& array_final, loginfo_t& loginfo){            
    // TODO: implementar    
}

Overwriting sorting4.cpp


In [ ]:
%%script bash
g++ sorting4.cpp -o sorting4

In [ ]:
%%script bash
./sorting4

## Exercício 3: Avaliação do desempenho do `merge_sort`

Utilizando o `merge_sort` que você implementou acima, faça uma análise de desempenho dele para arrays de tamanho 100, 1000 e 10000, tanto crescentes quanto decrescentes. Registre o tempo de processamento, a quantidade de trocas e a quantidade de comparações.


In [ ]:
%%writefile ms_test.cpp
// TODO: implementar

Overwriting ms_test.cpp


In [ ]:
%%script bash
g++ ms_test.cpp -o ms_test
./ms_test

## Parte IV: algoritmos de tempo linear

Implemente os algoritmos:

1. `RADIX-sort LSD` 
2. `RADIX-sort MSD` 
3. `CountingSort`

Depois, teste o seu desempenho (tempo de processmento) para ordenar um array decrescente de 100 elementos.